In [2]:
import org.apache.spark.sql.{DataFrame, Dataset, Row, SparkSession}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._
val spark = SparkSession.builder.master("local[*]").appName("dataProcessing").getOrCreate()
import spark.implicits._

spark = org.apache.spark.sql.SparkSession@4fd07a98


org.apache.spark.sql.SparkSession@4fd07a98

In [ ]:
import java.io.File
import scala.io.Source

val path = System.getProperty("user.dir")
val seasons = 1990 to 2018
val teams = 1 to 30

for (i <- seasons){
    for (j <- teams){
        val file = path + "/season_averages/" + i + "/rosters_old/" + j + ".json"
        try{
            var df_stats = spark.read.json(file)
            var elem = df_stats.select(explode($"data").as("data"))
            var minutes = elem.select("data.min")
            val split_min = elem.withColumn("seconds", split($"data.min", ":")(0) * 60 + split($"data.min", ":")(1))
            val best_12 = split_min.orderBy(desc("seconds")).limit(12)
            
            val best_12_struct = best_12.withColumn("row", struct(col("data.ast"), col("data.blk"), col("data.dreb"),
                                                                  col("data.fg3_pct"), col("data.fg3a"), 
                                                                  col("data.fg3m"), col("data.fg_pct"), 
                                                                  col("data.fga"), col("data.fgm"), 
                                                                  col("data.ft_pct"), col("data.fta"),
                                                                  col("data.ftm"), col("data.games_played"), 
                                                                  col("seconds"), col("data.oreb"), col("data.pf"),
                                                                  col("data.player_id"), 
                                                                  col("data.pts"), col("data.reb"), 
                                                                  col("data.season"), col("data.stl"), 
                                                                  col("data.turnover"))).agg(collect_list(col("row")).as("data"))
            
            val best_12_data = best_12_struct.select("data")
            best_12_data.write.mode("overwrite").format("json").save(path + "/season_averages/" + i + "/" + j)

            
            val best_12_36 = best_12.withColumn("ast", (col("data.ast") / col("seconds") * 2160))
                                    .withColumn("blk", col("data.blk") / col("seconds") * 2160)
                                    .withColumn("dreb", col("data.dreb") / col("seconds") * 2160)
                                    .withColumn("fg3a", col("data.fg3a") / col("seconds") * 2160)
                                    .withColumn("fg3m", col("data.fg3m") / col("seconds") * 2160)
                                    .withColumn("fga", col("data.fga") / col("seconds") * 2160)
                                    .withColumn("fgm", col("data.fgm") / col("seconds") * 2160)
                                    .withColumn("fta", col("data.fta") / col("seconds") * 2160)
                                    .withColumn("ftm", col("data.ftm") / col("seconds") * 2160)
                                    .withColumn("oreb", col("data.oreb") / col("seconds") * 2160)
                                    .withColumn("pf", col("data.pf") / col("seconds") * 2160)
                                    .withColumn("pts", col("data.pts") / col("seconds") * 2160)
                                    .withColumn("reb", col("data.reb") / col("seconds") * 2160)
                                    .withColumn("seconds", col("seconds") / col("seconds") * 2160)
                                    .withColumn("stl", col("data.stl") / col("seconds") * 2160)
                                    .withColumn("turnover", col("data.turnover") / col("seconds") * 2160)

            val best_12_36_struct = best_12_36.withColumn("row", struct(col("ast"), col("blk"), col("dreb"), 
                                                                        col("data.fg3_pct"), col("fg3a"), 
                                                                        col("fg3m"), col("data.fg_pct"), col("fga"),
                                                                        col("fgm"), col("data.ft_pct"),
                                                                        col("fta"), col("ftm"),
                                                                        col("data.games_played"), 
                                                                        col("seconds"), col("oreb"), 
                                                                        col("pf"), col("data.player_id"),
                                                                        col("pts"), col("reb"), col("data.season"),
                                                                        col("stl"), col("turnover"))).agg(collect_list(col("row")).as("data"))
            val best_12_36_data = best_12_36_struct.select("data")
            best_12_36_data.write.mode("overwrite").format("json").save(path + "/season_averages/" + i + "/" + j + "-36")
            println("File " + file + " saved")
            
        
        }
        catch{
            case e: org.apache.spark.sql.AnalysisException => println("File " + file + " not found")
        }
    } 
}

File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/1990/rosters_old/1.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/1990/rosters_old/2.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/1990/rosters_old/3.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/1990/rosters_old/4.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/1990/rosters_old/5.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/1990/rosters_old/6.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/1990/rosters_old/7.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/1990/rosters_old/8.json saved
File /home/adrian/Desktop/DataIn

File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/1992/rosters_old/9.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/1992/rosters_old/10.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/1992/rosters_old/11.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/1992/rosters_old/12.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/1992/rosters_old/13.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/1992/rosters_old/14.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/1992/rosters_old/15.json not found
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/1992/rosters_old/16.json saved
File /home/adrian/Des

File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/1994/rosters_old/17.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/1994/rosters_old/18.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/1994/rosters_old/19.json not found
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/1994/rosters_old/20.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/1994/rosters_old/21.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/1994/rosters_old/22.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/1994/rosters_old/23.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/1994/rosters_old/24.json saved
File /home/adrian/De

File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/1996/rosters_old/25.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/1996/rosters_old/26.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/1996/rosters_old/27.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/1996/rosters_old/28.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/1996/rosters_old/29.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/1996/rosters_old/30.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/1997/rosters_old/1.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/1997/rosters_old/2.json saved
File /home/adrian/Desktop/

File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/1999/rosters_old/3.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/1999/rosters_old/4.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/1999/rosters_old/5.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/1999/rosters_old/6.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/1999/rosters_old/7.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/1999/rosters_old/8.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/1999/rosters_old/9.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/1999/rosters_old/10.json saved
File /home/adrian/Desktop/DataI

File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2001/rosters_old/11.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2001/rosters_old/12.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2001/rosters_old/13.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2001/rosters_old/14.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2001/rosters_old/15.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2001/rosters_old/16.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2001/rosters_old/17.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2001/rosters_old/18.json saved
File /home/adrian/Deskto

File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2003/rosters_old/19.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2003/rosters_old/20.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2003/rosters_old/21.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2003/rosters_old/22.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2003/rosters_old/23.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2003/rosters_old/24.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2003/rosters_old/25.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2003/rosters_old/26.json saved
File /home/adrian/Deskto

File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2005/rosters_old/27.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2005/rosters_old/28.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2005/rosters_old/29.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2005/rosters_old/30.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2006/rosters_old/1.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2006/rosters_old/2.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2006/rosters_old/3.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2006/rosters_old/4.json saved
File /home/adrian/Desktop/Da

File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2008/rosters_old/5.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2008/rosters_old/6.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2008/rosters_old/7.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2008/rosters_old/8.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2008/rosters_old/9.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2008/rosters_old/10.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2008/rosters_old/11.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2008/rosters_old/12.json saved
File /home/adrian/Desktop/Dat

File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2010/rosters_old/13.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2010/rosters_old/14.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2010/rosters_old/15.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2010/rosters_old/16.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2010/rosters_old/17.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2010/rosters_old/18.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2010/rosters_old/19.json saved
File /home/adrian/Desktop/DataIntensive-Project-Prediction-BasketballGames/season_averages/2010/rosters_old/20.json saved
File /home/adrian/Deskto